In [ ]:
pip install python_speech_features

In [6]:
from python_speech_features import mfcc
from sklearn.mixture import GaussianMixture
import scipy as sc
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [7]:
dir = "/content/drive/MyDrive/Projet/Langues/Dataset/Russian"

In [9]:
folders_out = ["Train", "Test"]
folders_in = ["F", "H"]
mfcc_folder = "/content/drive/MyDrive/Projet/Langues/MFCC/Russian"

for folder_out in folders_out:
  for folder_in in folders_in:

    data_folder = dir + ("/") + folder_out + ("/") + folder_in
    audio_files = os.listdir(data_folder)
    audio_files = [filename for filename in audio_files if filename.endswith(('.wav'))]

    for audio_file in audio_files:
      
      basename = audio_file.split('.')[0]+("_mfcc")

      rate, data = sc.io.wavfile.read(os.path.join(data_folder, audio_file))
      mfcc_feat = mfcc(data, rate, nfft=2043)
      mfcc_feat_arr = np.asarray(mfcc_feat)

      path = mfcc_folder + ("/") + folder_out + ("/") + folder_in
      np.savetxt(path+("/")+basename+".txt", mfcc_feat_arr, delimiter=',')   

**Silence Removal**

In [ ]:
gmm = GaussianMixture(n_components=2, covariance_type='full')

for folder_out in folders_out:
  for folder_in in folders_in:

    data_folder = mfcc_folder + ("/") + folder_out + ("/") + folder_in
    mfcc_files = os.listdir(data_folder)
    mfcc_files = [filename for filename in mfcc_files if filename.endswith('txt')]

    for filename in mfcc_files :

      basename = filename.split('.')[0] + ("_no_silence")

      mfcc = np.loadtxt(os.path.join(data_folder, filename), delimiter=',')
      print("Avant : ", mfcc.shape)

      energies = np.sum(np.square(mfcc), axis=1)
      energies = energies.reshape(-1, 1)

      gmm.fit(energies)
      means = gmm.means_
      silence_component = np.argmin(means)
      silence_threshold = means[silence_component] - 2 * np.sqrt(gmm.covariances_[silence_component])
      
      non_silent_mask = energies > silence_threshold
      mfcc_no_silence = energies[non_silent_mask]
      print("Après : ", mfcc_no_silence.shape)

      path = mfcc_folder + ("/Preprocessing/") + folder_out + ("/") + folder_in
      np.savetxt(path+("/")+basename+".txt", mfcc_no_silence, delimiter=',')